In [3]:
pip install  httr

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement httr (from versions: none)

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for httr


In [6]:
#!/usr/bin/env python3
"""
🚀 ADVANCED ADA PORTFOLIO ANALYSIS SYSTEM - PYTHON VERSION
===========================================================
Cross-Platform Compatible: Python 3.12.3
Integration: Bitget API + Technical Analysis
Purpose: Comprehensive Portfolio Assessment
"""

import os
import sys
import json
import time
import hmac
import hashlib
import base64
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Any
import warnings
warnings.filterwarnings('ignore')

# Try to import technical analysis libraries
try:
    import yfinance as yf
    import talib
    TALIB_AVAILABLE = True
except ImportError:
    TALIB_AVAILABLE = False
    print("⚠️ Installing required packages...")
    os.system("pip install yfinance pandas numpy requests")

# ==========================================
# 🔧 BITGET API INTEGRATION
# ==========================================

class BitgetAPI:
    def __init__(self):
        self.api_key = os.getenv("BITGET_API_KEY")
        self.api_secret = os.getenv("BITGET_API_SECRET")
        self.passphrase = os.getenv("BITGET_PASSPHRASE")
        self.base_url = "https://api.bitget.com"
        
        if not all([self.api_key, self.api_secret, self.passphrase]):
            print("⚠️ Warning: API credentials not found in environment variables")
    
    def _generate_signature(self, timestamp: str, method: str, request_path: str, body: str = "") -> str:
        """Generate API signature"""
        message = timestamp + method + request_path + body
        mac = hmac.new(
            self.api_secret.encode('utf-8'),
            message.encode('utf-8'),
            hashlib.sha256
        )
        return base64.b64encode(mac.digest()).decode('utf-8')
    
    def _make_request(self, endpoint: str, method: str = "GET", params: Dict = None) -> Optional[Dict]:
        """Make authenticated API request"""
        try:
            timestamp = str(int(time.time() * 1000))
            
            if method == "GET" and params:
                query_string = "&".join([f"{k}={v}" for k, v in params.items()])
                request_path = f"{endpoint}?{query_string}"
            else:
                request_path = endpoint
                query_string = ""
            
            body = json.dumps(params) if method == "POST" and params else ""
            signature = self._generate_signature(timestamp, method, request_path, body)
            
            headers = {
                "ACCESS-KEY": self.api_key,
                "ACCESS-SIGN": signature,
                "ACCESS-TIMESTAMP": timestamp,
                "ACCESS-PASSPHRASE": self.passphrase,
                "Content-Type": "application/json"
            }
            
            url = self.base_url + request_path
            
            if method == "GET":
                response = requests.get(url, headers=headers, timeout=10)
            else:
                response = requests.post(url, headers=headers, data=body, timeout=10)
            
            response.raise_for_status()
            return response.json()
            
        except Exception as e:
            print(f"❌ API Error: {e}")
            return None
    
    def get_account_data(self) -> Optional[Dict]:
        """Fetch account data"""
        print("🔍 Fetching account data...")
        result = self._make_request("/api/mix/v1/account/accounts", params={"productType": "umcbl"})
        
        if result and result.get("code") == "00000":
            return result.get("data")
        return None
    
    def get_positions(self) -> Optional[List[Dict]]:
        """Fetch all positions"""
        print("🔍 Fetching position data...")
        result = self._make_request("/api/mix/v1/position/allPosition", params={"productType": "umcbl"})
        
        if result and result.get("code") == "00000":
            positions = result.get("data", [])
            # Filter for active ADA positions
            ada_positions = [
                pos for pos in positions 
                if "ADA" in pos.get("symbol", "").upper() and float(pos.get("total", 0)) > 0
            ]
            return ada_positions
        return []

# ==========================================
# 📊 MARKET DATA & TECHNICAL ANALYSIS
# ==========================================

class MarketAnalysis:
    def __init__(self):
        self.symbol = "ADA-USD"
    
    def get_market_data(self, days: int = 100) -> Optional[pd.DataFrame]:
        """Fetch market data from Yahoo Finance"""
        print(f"🔍 Fetching market data for {self.symbol}...")
        
        try:
            end_date = datetime.now()
            start_date = end_date - timedelta(days=days)
            
            ticker = yf.Ticker(self.symbol)
            data = ticker.history(start=start_date, end=end_date)
            
            if data.empty:
                print("⚠️ No market data available, generating mock data...")
                return self._generate_mock_data(days)
            
            return data
            
        except Exception as e:
            print(f"⚠️ Market data error: {e}, using mock data")
            return self._generate_mock_data(days)
    
    def _generate_mock_data(self, days: int) -> pd.DataFrame:
        """Generate mock market data for testing"""
        dates = pd.date_range(start=datetime.now() - timedelta(days=days), 
                             end=datetime.now(), freq='D')
        
        # Generate realistic ADA price movement
        np.random.seed(42)
        base_price = 0.65
        returns = np.random.normal(0, 0.02, len(dates))
        prices = [base_price]
        
        for ret in returns[1:]:
            prices.append(prices[-1] * (1 + ret))
        
        data = pd.DataFrame({
            'Open': prices,
            'High': [p * (1 + abs(np.random.normal(0, 0.01))) for p in prices],
            'Low': [p * (1 - abs(np.random.normal(0, 0.01))) for p in prices],
            'Close': prices,
            'Volume': np.random.uniform(1000000, 5000000, len(dates))
        }, index=dates)
        
        return data
    
    def calculate_indicators(self, data: pd.DataFrame) -> Dict[str, float]:
        """Calculate technical indicators"""
        print("📊 Calculating technical indicators...")
        
        if data.empty:
            return {}
        
        close = data['Close'].values
        high = data['High'].values
        low = data['Low'].values
        volume = data['Volume'].values
        
        indicators = {}
        
        try:
            # Basic indicators
            indicators['current_price'] = float(close[-1])
            indicators['sma_20'] = float(np.mean(close[-20:]))
            indicators['sma_50'] = float(np.mean(close[-50:]) if len(close) >= 50 else np.mean(close))
            
            # RSI calculation (simplified)
            def calculate_rsi(prices, period=14):
                deltas = np.diff(prices)
                seed = deltas[:period+1]
                up = seed[seed >= 0].sum() / period
                down = -seed[seed < 0].sum() / period
                rs = up / down if down != 0 else 0
                rsi = 100. - 100. / (1. + rs)
                
                for i in range(period, len(deltas)):
                    delta = deltas[i-1]
                    if delta > 0:
                        upval = delta
                        downval = 0.
                    else:
                        upval = 0.
                        downval = -delta
                    
                    up = (up * (period - 1) + upval) / period
                    down = (down * (period - 1) + downval) / period
                    rs = up / down if down != 0 else 0
                    rsi = 100. - 100. / (1. + rs)
                
                return rsi
            
            indicators['rsi_14'] = calculate_rsi(close)
            
            # EMA calculation
            def calculate_ema(prices, period):
                multiplier = 2 / (period + 1)
                ema = [prices[0]]
                for price in prices[1:]:
                    ema.append((price * multiplier) + (ema[-1] * (1 - multiplier)))
                return ema[-1]
            
            indicators['ema_12'] = calculate_ema(close, 12)
            indicators['ema_26'] = calculate_ema(close, 26)
            
            # MACD
            macd_line = indicators['ema_12'] - indicators['ema_26']
            indicators['macd'] = macd_line
            
            # Bollinger Bands
            bb_period = 20
            bb_std = 2
            if len(close) >= bb_period:
                sma = np.mean(close[-bb_period:])
                std = np.std(close[-bb_period:])
                indicators['bb_upper'] = sma + (bb_std * std)
                indicators['bb_lower'] = sma - (bb_std * std)
            
            # Volatility
            if len(close) >= 20:
                returns = np.diff(np.log(close[-20:]))
                indicators['volatility'] = float(np.std(returns) * np.sqrt(252))
            
            print(f"💰 Current Price: {indicators['current_price']:.6f} USDT")
            print(f"📊 RSI (14): {indicators['rsi_14']:.2f}")
            print(f"📈 SMA 20: {indicators['sma_20']:.6f} USDT")
            print(f"📈 SMA 50: {indicators['sma_50']:.6f} USDT")
            
        except Exception as e:
            print(f"⚠️ Error calculating indicators: {e}")
        
        return indicators

# ==========================================
# 🎯 PORTFOLIO ANALYSIS ENGINE
# ==========================================

class PortfolioAnalyzer:
    def __init__(self):
        self.api = BitgetAPI()
        self.market = MarketAnalysis()
    
    def analyze_portfolio_health(self, account_data: List[Dict]) -> Dict[str, Any]:
        """Analyze overall portfolio health"""
        print("\n📊 PORTFOLIO HEALTH ANALYSIS")
        print("=" * 40)
        
        if not account_data:
            print("❌ No account data available")
            return {}
        
        # Use first account (main account)
        account = account_data[0] if isinstance(account_data, list) else account_data
        
        equity = float(account.get('equity', 0))
        available = float(account.get('available', 0))
        unrealized_pnl = float(account.get('unrealizedPL', 0))
        locked = float(account.get('locked', 0))
        
        # Calculate metrics
        equity_utilization = (locked / equity * 100) if equity > 0 else 0
        pnl_percentage = (unrealized_pnl / equity * 100) if equity > 0 else 0
        available_percentage = (available / equity * 100) if equity > 0 else 0
        
        # Health score calculation
        health_score = 100
        risk_factors = []
        
        if equity_utilization > 80:
            health_score -= 30
            risk_factors.append("High leverage risk")
        
        if pnl_percentage < -10:
            health_score -= 25
            risk_factors.append("Significant losses")
        
        if available_percentage < 10:
            health_score -= 20
            risk_factors.append("Low available margin")
        
        # Display results
        print(f"💰 Total Equity: {equity:.4f} USDT")
        print(f"💵 Available: {available:.4f} USDT ({available_percentage:.1f}%)")
        print(f"🔒 Locked: {locked:.4f} USDT ({equity_utilization:.1f}%)")
        print(f"📈 Unrealized P&L: {unrealized_pnl:.4f} USDT ({pnl_percentage:.2f}%)")
        print(f"⚡ Health Score: {health_score:.0f}/100")
        
        if risk_factors:
            print("⚠️ Risk Factors:")
            for factor in risk_factors:
                print(f"   • {factor}")
        
        return {
            'equity': equity,
            'available': available,
            'locked': locked,
            'unrealized_pnl': unrealized_pnl,
            'equity_utilization': equity_utilization,
            'pnl_percentage': pnl_percentage,
            'health_score': health_score,
            'risk_factors': risk_factors
        }
    
    def analyze_ada_positions(self, positions: List[Dict], market_data: pd.DataFrame) -> List[Dict]:
        """Analyze ADA positions"""
        print("\n🎯 ADA POSITION ANALYSIS")
        print("=" * 40)
        
        if not positions:
            print("📭 No active ADA positions found")
            return []
        
        current_price = market_data['Close'].iloc[-1] if not market_data.empty else 0.65
        position_analysis = []
        
        for i, pos in enumerate(positions):
            symbol = pos.get('symbol', '')
            side = pos.get('holdSide', '')
            size = float(pos.get('total', 0))
            avg_price = float(pos.get('averageOpenPrice', 0))
            mark_price = float(pos.get('markPrice', current_price))
            unrealized_pnl = float(pos.get('unrealizedPL', 0))
            margin = float(pos.get('margin', 0))
            leverage = float(pos.get('leverage', 1))
            liquidation_price = float(pos.get('liquidationPrice', 0))
            
            # Calculations
            price_change_pct = ((current_price - avg_price) / avg_price * 100) if avg_price > 0 else 0
            roe_pct = (unrealized_pnl / margin * 100) if margin > 0 else 0
            
            if side == "long":
                distance_to_liquidation = ((current_price - liquidation_price) / current_price * 100) if current_price > 0 else 0
            else:
                distance_to_liquidation = ((liquidation_price - current_price) / current_price * 100) if current_price > 0 else 0
            
            position_value = size * current_price
            
            # Risk assessment
            risk_level = "LOW"
            if abs(roe_pct) > 50 or distance_to_liquidation < 20:
                risk_level = "HIGH"
            elif abs(roe_pct) > 25 or distance_to_liquidation < 50:
                risk_level = "MEDIUM"
            
            print(f"\n🔸 Position: {symbol}")
            print(f"   Side: {'🟢' if side == 'long' else '🔴'} {side.upper()}")
            print(f"   Size: {size} ADA ({position_value:.2f} USDT)")
            print(f"   Avg Price: {avg_price:.6f} USDT")
            print(f"   Current Price: {current_price:.6f} USDT")
            print(f"   Price Change: {price_change_pct:.2f}%")
            print(f"   Unrealized P&L: {unrealized_pnl:.4f} USDT")
            print(f"   ROE: {roe_pct:.2f}%")
            print(f"   Leverage: {leverage}x")
            print(f"   Margin: {margin:.4f} USDT")
            print(f"   Liquidation: {liquidation_price:.6f} USDT ({distance_to_liquidation:.1f}% away)")
            print(f"   Risk Level: {risk_level}")
            
            analysis = {
                'symbol': symbol,
                'side': side,
                'size': size,
                'avg_price': avg_price,
                'current_price': current_price,
                'unrealized_pnl': unrealized_pnl,
                'roe_pct': roe_pct,
                'leverage': leverage,
                'margin': margin,
                'risk_level': risk_level,
                'distance_to_liquidation': distance_to_liquidation,
                'position_value': position_value
            }
            
            position_analysis.append(analysis)
        
        return position_analysis
    
    def generate_trading_signals(self, indicators: Dict, position_analysis: List[Dict]) -> Dict[str, Any]:
        """Generate trading signals based on technical analysis"""
        print("\n🎯 TRADING SIGNALS")
        print("=" * 40)
        
        if not indicators:
            print("❌ No indicators available for signal generation")
            return {}
        
        signal_strength = 0
        signal_direction = "HOLD"
        reasons = []
        
        # RSI Signals
        rsi = indicators.get('rsi_14', 50)
        if rsi < 30:
            signal_strength += 2
            reasons.append("RSI Oversold")
        elif rsi > 70:
            signal_strength -= 2
            reasons.append("RSI Overbought")
        
        # Moving Average Signals
        current_price = indicators.get('current_price', 0)
        sma_20 = indicators.get('sma_20', 0)
        sma_50 = indicators.get('sma_50', 0)
        
        if current_price > sma_20 > sma_50:
            signal_strength += 1
            reasons.append("Bullish MA Cross")
        elif current_price < sma_20 < sma_50:
            signal_strength -= 1
            reasons.append("Bearish MA Cross")
        
        # MACD Signals
        macd = indicators.get('macd', 0)
        if macd > 0:
            signal_strength += 1
            reasons.append("MACD Bullish")
        else:
            signal_strength -= 1
            reasons.append("MACD Bearish")
        
        # Bollinger Band Signals
        bb_upper = indicators.get('bb_upper', 0)
        bb_lower = indicators.get('bb_lower', 0)
        
        if bb_lower > 0 and current_price <= bb_lower:
            signal_strength += 1
            reasons.append("BB Oversold")
        elif bb_upper > 0 and current_price >= bb_upper:
            signal_strength -= 1
            reasons.append("BB Overbought")
        
        # Determine final signal
        if signal_strength >= 2:
            signal_direction = "STRONG BUY"
        elif signal_strength >= 1:
            signal_direction = "BUY"
        elif signal_strength <= -2:
            signal_direction = "STRONG SELL"
        elif signal_strength <= -1:
            signal_direction = "SELL"
        else:
            signal_direction = "HOLD"
        
        confidence = min(abs(signal_strength) * 20, 90)
        
        print(f"🎯 Signal: {signal_direction}")
        print(f"💪 Confidence: {confidence}%")
        print(f"📊 Signal Strength: {signal_strength}")
        print("📋 Reasons:")
        for reason in reasons:
            print(f"   • {reason}")
        
        return {
            'signal': signal_direction,
            'confidence': confidence,
            'strength': signal_strength,
            'reasons': reasons,
            'timestamp': datetime.now().isoformat()
        }
    
    def generate_recommendations(self, portfolio_health: Dict, position_analysis: List[Dict], signals: Dict) -> List[str]:
        """Generate actionable recommendations"""
        print("\n💡 PORTFOLIO RECOMMENDATIONS")
        print("=" * 40)
        
        recommendations = []
        
        # Portfolio health recommendations
        if portfolio_health:
            if portfolio_health.get('health_score', 100) < 70:
                recommendations.append("🚨 Consider reducing leverage or position size")
            
            if portfolio_health.get('equity_utilization', 0) > 80:
                recommendations.append("⚠️ Add more margin to reduce liquidation risk")
            
            if portfolio_health.get('pnl_percentage', 0) < -5:
                recommendations.append("📉 Consider stop-loss strategy to limit losses")
        
        # Position-specific recommendations
        for pos in position_analysis:
            if pos.get('risk_level') == "HIGH":
                recommendations.append(f"🔴 {pos['symbol']} position at HIGH risk - consider partial close")
            
            if pos.get('distance_to_liquidation', 100) < 30:
                recommendations.append(f"⚡ {pos['symbol']} close to liquidation - add margin or reduce size")
            
            if pos.get('roe_pct', 0) > 20:
                recommendations.append(f"💰 {pos['symbol']} in profit - consider taking partial profits")
        
        # Signal-based recommendations
        if signals:
            signal = signals.get('signal', '')
            confidence = signals.get('confidence', 0)
            
            if signal == "STRONG BUY" and confidence > 70:
                recommendations.append("🚀 Strong buy signal - consider increasing position")
            elif signal == "STRONG SELL" and confidence > 70:
                recommendations.append("📉 Strong sell signal - consider reducing position")
        
        # General recommendations
        recommendations.extend([
            "📊 Monitor RSI for entry/exit points",
            "🎯 Set stop-loss orders for risk management", 
            "📈 Consider DCA strategy for volatile periods"
        ])
        
        if not recommendations:
            recommendations.append("✅ Portfolio looks healthy - continue monitoring")
        
        for i, rec in enumerate(recommendations, 1):
            print(f"{i}. {rec}")
        
        return recommendations
    
    def save_analysis_report(self, analysis_data: Dict) -> bool:
        """Save analysis report to JSON files"""
        print("\n💾 SAVING ANALYSIS REPORT")
        print("=" * 40)
        
        try:
            # Create output directory
            output_dir = "../shared_data"
            os.makedirs(output_dir, exist_ok=True)
            
            # Save full analysis report
            report_file = os.path.join(output_dir, "portfolio_analysis.json")
            with open(report_file, 'w') as f:
                json.dump(analysis_data, f, indent=2, default=str)
            print(f"✅ Full report saved to: {report_file}")
            
            # Save signals for Python bot integration
            if 'signals' in analysis_data and analysis_data['signals']:
                signals_file = os.path.join(output_dir, "r_signals.json")
                with open(signals_file, 'w') as f:
                    json.dump(analysis_data['signals'], f, indent=2, default=str)
                print(f"✅ Signals saved to: {signals_file}")
            
            return True
            
        except Exception as e:
            print(f"❌ Failed to save report: {e}")
            return False

# ==========================================
# 🚀 MAIN ANALYSIS EXECUTION
# ==========================================

def main_portfolio_analysis():
    """Main portfolio analysis function"""
    print("🚀 STARTING COMPREHENSIVE PORTFOLIO ANALYSIS")
    print("=" * 50)
    print(f"📅 Timestamp: {datetime.now()}")
    print(f"🐍 Python Version: {sys.version.split()[0]}")
    print(f"📍 Working Directory: {os.getcwd()}")
    
    analyzer = PortfolioAnalyzer()
    
    try:
        # Step 1: Collect data
        account_data = analyzer.api.get_account_data()
        position_data = analyzer.api.get_positions()
        market_data = analyzer.market.get_market_data()
        
        # Step 2: Calculate technical indicators
        indicators = analyzer.market.calculate_indicators(market_data)
        
        # Step 3: Analyze portfolio health
        portfolio_health = analyzer.analyze_portfolio_health(account_data)
        
        # Step 4: Analyze positions
        position_analysis = analyzer.analyze_ada_positions(position_data, market_data)
        
        # Step 5: Generate trading signals
        signals = analyzer.generate_trading_signals(indicators, position_analysis)
        
        # Step 6: Generate recommendations
        recommendations = analyzer.generate_recommendations(portfolio_health, position_analysis, signals)
        
        # Step 7: Compile and save results
        analysis_data = {
            'timestamp': datetime.now().isoformat(),
            'portfolio_health': portfolio_health,
            'position_analysis': position_analysis,
            'technical_indicators': indicators,
            'signals': signals,
            'recommendations': recommendations
        }
        
        analyzer.save_analysis_report(analysis_data)
        
        print("\n🎉 PORTFOLIO ANALYSIS COMPLETE")
        print("=" * 40)
        
        return analysis_data
        
    except Exception as e:
        print(f"❌ Analysis failed: {e}")
        return None

if __name__ == "__main__":
    # Set up API credentials if not already set
    if not os.getenv("BITGET_API_KEY"):
        print("⚠️ Setting up demo mode (no API credentials found)")
        os.environ["BITGET_API_KEY"] = "demo"
        os.environ["BITGET_API_SECRET"] = "demo" 
        os.environ["BITGET_PASSPHRASE"] = "demo"
    
    # Run the analysis
    result = main_portfolio_analysis()
    
    if result:
        print("\n✅ Analysis completed successfully!")
        print("📊 Check the generated reports in ../shared_data/")
    else:
        print("\n❌ Analysis failed. Check the error messages above.")

⚠️ Installing required packages...
🚀 STARTING COMPREHENSIVE PORTFOLIO ANALYSIS
📅 Timestamp: 2025-06-20 19:38:38.402453
🐍 Python Version: 3.11.8
📍 Working Directory: c:\freeding\tbot202506\r_analysis\strategies\Bitget\Python
🔍 Fetching account data...
🔍 Fetching position data...
🔍 Fetching market data for ADA-USD...
⚠️ Market data error: name 'yf' is not defined, using mock data
📊 Calculating technical indicators...
💰 Current Price: 0.499707 USDT
📊 RSI (14): 42.70
📈 SMA 20: 0.523945 USDT
📈 SMA 50: 0.524943 USDT

📊 PORTFOLIO HEALTH ANALYSIS
💰 Total Equity: 830.7581 USDT
💵 Available: 1079.6652 USDT (130.0%)
🔒 Locked: 0.0000 USDT (0.0%)
📈 Unrealized P&L: -248.9070 USDT (-29.96%)
⚡ Health Score: 75/100
⚠️ Risk Factors:
   • Significant losses

🎯 ADA POSITION ANALYSIS

🔸 Position: ADAUSDT_UMCBL
   Side: 🟢 LONG
   Size: 5000.0 ADA (2498.53 USDT)
   Avg Price: 0.612581 USDT
   Current Price: 0.499707 USDT
   Price Change: -18.43%
   Unrealized P&L: -248.9070 USDT
   ROE: -81.26%
   Leverage: 1